# TF Encrypted Private Prediction

In [1]:
import syft
import numpy as np
import tensorflow as tf

## Step 1: Train model with Plaintext Keras

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, AveragePooling2D
from keras.layers import ReLU

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

public_model = Sequential()

public_model.add(Conv2D(10, (3, 3), input_shape=input_shape))
public_model.add(AveragePooling2D((2, 2)))
public_model.add(ReLU())
public_model.add(Conv2D(32, (3, 3)))
public_model.add(AveragePooling2D((2, 2)))
public_model.add(ReLU())
public_model.add(Conv2D(64, (3, 3)))
public_model.add(AveragePooling2D((2, 2)))
public_model.add(ReLU())
public_model.add(Flatten())
public_model.add(Dense(num_classes, activation='softmax'))

public_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

public_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = public_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 7s 113us/step - loss: 0.6592 - acc: 0.7916 - val_loss: 0.2570 - val_acc: 0.9186
Epoch 2/2
60000/60000 [==============================] - 7s 109us/step - loss: 0.2349 - acc: 0.9293 - val_loss: 0.2035 - val_acc: 0.9359
Test loss: 0.2034820306122303
Test accuracy: 0.9359


In [3]:
# Save model weights
weights_from_model_owner = public_model.get_weights()

## Private Prediction using Syft Keras

In [4]:
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU

In [5]:
task_shape = [None, 28, 28, 1]

In [6]:
# Define private model
# Should be equvalent to the plaintext model
private_model = Sequential()

private_model.add(Conv2D(10, (3, 3), input_shape=task_shape[1:]))
private_model.add(AveragePooling2D((2, 2)))
private_model.add(ReLU())
private_model.add(Conv2D(32, (3, 3)))
private_model.add(AveragePooling2D((2, 2)))
private_model.add(ReLU())
private_model.add(Conv2D(64, (3, 3)))
private_model.add(AveragePooling2D((2, 2)))
private_model.add(ReLU())
private_model.add(Flatten())
private_model.add(Dense(10, name="logit"))

In [7]:
# Load pre-trained model 
private_model.set_weights(weights_from_model_owner)

In [8]:
# prot = tfe.SecureNN(player_0=alice,
#                     player_1=bob,
#                     comparison_helper=carol,
#                     crypto_producer=carol)

# Currently using keras under the hood.
# will have to provide protocol when tfe.keras is ready
# the share method instantiate the keras model and load the weights
private_model_secured = private_model.share(prot=None)

In [9]:
# will have to provide data using x = tfe.define_private_input
data = np.ones([1, 28, 28, 1])

# Compute private prediction. Will have to use `tfe.define_output`
private_prediction = private_model_secured.predict(data)

print(private_prediction)

[[  8.462496    -9.085601    10.706078     4.7028933   -0.02719639
   -0.5176338   15.803772   -13.880548     9.9395485   -6.1211925 ]]


In [10]:
from scipy.special import softmax

np.testing.assert_allclose(softmax(private_prediction), public_model.predict(data), rtol=0.001)